In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [4]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [5]:
def getValidatorInfo(validator):
    method = "hmy_getValidatorInformation"
    params = [validator]
    return get_information(method, params)['result']

In [6]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result']

In [7]:
block = getBlockNumber()
last_block = getLastBlockOfCurrentEpoch()
print("current and last block", block, last_block)
num = last_block - block + 5
iterations = 0
while iterations < num:
    print('current block:', block)
    validator_infos = getAllValidatorInformation()
    epos_status = []
    boot_status = []
    address = []
    for info in validator_infos:    
        epos_status.append(info['epos-status'])
        address.append(info['validator']['address'])
        boot_status.append(info['booted-status'])
    df = pd.DataFrame(list(zip(address, epos_status, boot_status)), columns = ['address','epos-status','boot-status'])
    print(df[df['boot-status'].isna()].groupby('epos-status')['epos-status'].count())
    new_block = block + 1
    while block < new_block:
        block = getBlockNumber()
    iterations += 1


current and last block 108995 109021
current block: 108995
epos-status
currently elected                        193
eligible to be elected next epoch         80
not eligible to be elected next epoch      6
Name: epos-status, dtype: int64
current block: 108996
epos-status
currently elected                        193
eligible to be elected next epoch         80
not eligible to be elected next epoch      6
Name: epos-status, dtype: int64
current block: 108997
epos-status
currently elected                        193
eligible to be elected next epoch         80
not eligible to be elected next epoch      6
Name: epos-status, dtype: int64
current block: 108998
epos-status
currently elected                        193
eligible to be elected next epoch         80
not eligible to be elected next epoch      6
Name: epos-status, dtype: int64
current block: 108999
epos-status
currently elected                        193
eligible to be elected next epoch         80
not eligible to be elected next epo

In [12]:
df.groupby('epos-status')['epos-status'].count()

epos-status
currently elected                        193
eligible to be elected next epoch        208
not eligible to be elected next epoch    202
Name: epos-status, dtype: int64

In [17]:
address = df[(df['boot-status'].isna()) & (df['epos-status'] == 'eligible to be elected next epoch')].address.tolist()

In [26]:
for i in address:
    if getValidatorInfo(i)['lifetime']['blocks']['signed'] != 0:
        print(i)

In [20]:
no_address = df[(df['boot-status'].isna()) & (df['epos-status'] == 'not eligible to be elected next epoch')].address.tolist()

In [25]:
for i in no_address:
    if getValidatorInfo(i)['lifetime']['blocks']['signed'] != 0:
        print(i)

In [24]:
getValidatorInfo(i)['lifetime']['blocks']['signed']


{'to-sign': 0, 'signed': 0}